In [85]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import datetime
import os

In [39]:
trends = pd.read_csv("../csv/trends/input.csv")
options = pd.read_csv("../csv/options/raw/2015-16options.csv")

In [40]:
pruned_trends = trends
pruned_trends["date"] = pd.to_datetime(trends["date"])
pruned_trends.head()

,date,0,1,2,3,4,5,6,7,8,...,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974
0,2015-01-02 09:00:00,37.0,30.0,55.0,49.0,18.0,47.0,22.0,30.0,12.0,...,47.0,35.0,60.0,57.0,29.0,40.0,29.0,0.0,34.0,60.0
1,2015-01-03 09:00:00,47.0,53.0,95.0,39.0,29.0,40.0,23.0,42.0,12.0,...,43.0,32.0,56.0,42.0,29.0,39.0,15.0,0.0,39.0,57.0
2,2015-01-04 09:00:00,41.0,33.0,65.0,41.0,28.0,49.0,38.0,39.0,11.0,...,38.0,25.0,51.0,29.0,31.0,38.0,13.0,0.0,39.0,60.0
3,2015-01-05 09:00:00,38.0,49.0,43.0,41.0,19.0,71.0,30.0,36.0,5.0,...,42.0,38.0,93.0,70.0,34.0,51.0,22.0,0.0,30.0,51.0
4,2015-01-06 09:00:00,59.0,38.0,100.0,45.0,21.0,53.0,23.0,57.0,6.0,...,47.0,34.0,86.0,77.0,34.0,56.0,25.0,0.0,30.0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,2021-07-13 09:00:00,65.0,0.0,92.0,22.0,41.0,0.0,40.0,64.0,35.0,...,55.0,64.0,72.0,63.0,22.0,0.0,57.0,25.0,42.0,0.0
2385,2021-07-14 09:00:00,66.0,0.0,85.0,25.0,67.0,0.0,39.0,65.0,36.0,...,57.0,65.0,82.0,72.0,22.0,0.0,68.0,14.0,43.0,0.0
2386,2021-07-15 09:00:00,70.0,0.0,98.0,22.0,54.0,0.0,37.0,68.0,33.0,...,54.0,0.0,70.0,67.0,19.0,84.0,55.0,15.0,40.0,0.0
2387,2021-07-16 09:00:00,64.0,0.0,87.0,21.0,25.0,40.0,40.0,72.0,27.0,...,52.0,0.0,65.0,73.0,16.0,78.0,62.0,15.0,36.0,0.0


In [41]:
pruned_trends["date"] = pruned_trends["date"].apply(lambda x: pd.Timestamp(x.date()))
pruned_trends = pruned_trends.set_index("date")

In [42]:
pruned_trends.head()

,0,1,2,3,4,5,6,7,8,9,...,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.0,30.0,55.0,49.0,18.0,47.0,22.0,30.0,12.0,4.0,...,47.0,35.0,60.0,57.0,29.0,40.0,29.0,0.0,34.0,60.0
2015-01-03,47.0,53.0,95.0,39.0,29.0,40.0,23.0,42.0,12.0,18.0,...,43.0,32.0,56.0,42.0,29.0,39.0,15.0,0.0,39.0,57.0
2015-01-04,41.0,33.0,65.0,41.0,28.0,49.0,38.0,39.0,11.0,21.0,...,38.0,25.0,51.0,29.0,31.0,38.0,13.0,0.0,39.0,60.0
2015-01-05,38.0,49.0,43.0,41.0,19.0,71.0,30.0,36.0,5.0,8.0,...,42.0,38.0,93.0,70.0,34.0,51.0,22.0,0.0,30.0,51.0
2015-01-06,59.0,38.0,100.0,45.0,21.0,53.0,23.0,57.0,6.0,10.0,...,47.0,34.0,86.0,77.0,34.0,56.0,25.0,0.0,30.0,53.0


In [4]:
options.head()

,secid,date,exdate,cp_flag,strike_price,volume,open_interest,impl_volatility,opprc,moneyness,tte,close,spread,noi
0,5284,01/02/2015,03/20/2015,C,2500,0,5,1.212510,0.100,1.677852,77,1.49,0.10,5
1,5284,01/02/2015,03/20/2015,P,2500,0,10,1.389238,1.150,1.677852,77,1.49,0.50,10
2,5284,01/05/2015,03/20/2015,C,2500,0,5,1.322124,0.100,1.760563,74,1.42,0.10,0
3,5284,01/05/2015,03/20/2015,P,2500,0,10,1.533191,1.225,1.760563,74,1.42,0.45,0
4,5284,01/06/2015,03/20/2015,C,2500,0,5,1.369239,0.100,1.798561,73,1.39,0.10,0


In [54]:
pruned_options = options[options["volume"] > 0]
pruned_options = pruned_options.drop(["secid", "opprc", "moneyness", "close", "spread", "noi", "open_interest", "exdate", "volume"], axis=1)
pruned_options = pd.get_dummies(pruned_options, columns=["cp_flag"])
pruned_options["date"] = pd.to_datetime(pruned_options["date"])
pruned_options = pruned_options.set_index("date")

In [55]:
pruned_options.head()

,strike_price,impl_volatility,tte,cp_flag_C,cp_flag_P
date,,,,,
2015-01-07,2500,1.378718,72,1,0
2015-04-27,2500,0.920548,144,1,0
2015-01-05,2500,0.808105,193,1,0
2015-01-07,2500,0.874225,191,1,0
2015-01-14,2500,0.909522,93,1,0


In [56]:
sample_size = 100000
sampled_options = pruned_options.sample(sample_size)

In [57]:
sampled_options.head()

,strike_price,impl_volatility,tte,cp_flag_C,cp_flag_P
date,,,,,
2016-12-27,73000,0.362538,38,0,1
2015-06-10,100000,0.403758,219,0,1
2016-04-25,29000,0.392765,81,0,1
2016-10-17,44000,0.439963,18,1,0
2015-03-20,27000,0.152658,42,1,0


In [58]:
input_df = sampled_options.apply(lambda x: pd.concat([x, pruned_trends.loc[pd.Timestamp(x.name)]], axis=0), axis=1)

In [59]:
input_df.head()

,strike_price,impl_volatility,tte,cp_flag_C,cp_flag_P,0,1,2,3,4,...,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974
date,,,,,,,,,,,,,,,,,,,,,
2016-12-27,73000.0,0.362538,38.0,0.0,1.0,51.0,33.0,73.0,23.0,12.0,...,35.0,45.0,76.0,62.0,14.0,49.0,71.0,22.0,43.0,53.0
2015-06-10,100000.0,0.403758,219.0,0.0,1.0,67.0,53.0,90.0,21.0,33.0,...,48.0,33.0,89.0,16.0,19.0,52.0,58.0,0.0,42.0,53.0
2016-04-25,29000.0,0.392765,81.0,0.0,1.0,39.0,33.0,43.0,16.0,33.0,...,39.0,42.0,89.0,65.0,24.0,52.0,52.0,9.0,33.0,43.0
2016-10-17,44000.0,0.439963,18.0,1.0,0.0,34.0,26.0,40.0,12.0,25.0,...,45.0,32.0,87.0,73.0,23.0,52.0,56.0,0.0,38.0,45.0
2015-03-20,27000.0,0.152658,42.0,1.0,0.0,64.0,43.0,89.0,18.0,29.0,...,39.0,36.0,69.0,54.0,17.0,43.0,49.0,0.0,37.0,48.0


In [60]:
input_df.shape

(100000, 1980)

In [61]:
y = input_df["impl_volatility"].to_numpy()

In [62]:
x = input_df.drop(["impl_volatility"], axis=1).to_numpy()

In [63]:
x.shape

(100000, 1979)

In [107]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1979, activation='relu'),
    tf.keras.layers.Dense(1979*2, activation='relu'),
    tf.keras.layers.Dense(1979*2, activation='relu'),
    tf.keras.layers.Dense(1979, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear'),
])

In [108]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss=keras.losses.MeanSquaredError(),
    metrics=[
        keras.metrics.MeanAbsolutePercentageError(),
        keras.metrics.MeanAbsoluteError()
             ]
)

In [114]:
model.fit(
    x,
    y,
    batch_size=64,
    epochs=12,
    shuffle=True,
    verbose=1,
    use_multiprocessing=True,
    workers=12,
    validation_split=0.25
)

Epoch 1/12
1407/1407 [==============================] - 194s 138ms/step - loss: 0.0534 - mean_absolute_percentage_error: 40.0509 - mean_absolute_error: 0.1465 - val_loss: 0.0492 - val_mean_absolute_percentage_error: 40.8452 - val_mean_absolute_error: 0.1452
Epoch 2/12
1407/1407 [==============================] - 191s 136ms/step - loss: 0.0535 - mean_absolute_percentage_error: 40.0422 - mean_absolute_error: 0.1461 - val_loss: 0.0491 - val_mean_absolute_percentage_error: 38.6027 - val_mean_absolute_error: 0.1412
Epoch 3/12
1407/1407 [==============================] - 192s 137ms/step - loss: 9.5411 - mean_absolute_percentage_error: 93.1041 - mean_absolute_error: 0.3017 - val_loss: 0.0496 - val_mean_absolute_percentage_error: 39.9250 - val_mean_absolute_error: 0.1448
Epoch 4/12
1407/1407 [==============================] - 195s 139ms/step - loss: 0.0537 - mean_absolute_percentage_error: 39.9624 - mean_absolute_error: 0.1458 - val_loss: 0.0509 - val_mean_absolute_percentage_error: 37.5687 - 

In [115]:
model.predict(x[[10, 20]])

array([[0.36470494],
       [0.36470494]], dtype=float32)

In [117]:
y[[10, 20]]

array([0.220851, 0.283556])

In [118]:
model.save("initial_model")

2022-03-10 17:08:09.445306: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: initial_model/assets
